In [1]:
# Import python libraries
#
import importlib
import pickle
import os
import torch

# Go one directory back, because all imports are done
# relative to the root of the project.
#
project_root = '..'
if 'change_directory_to_root' not in globals():
    change_directory_to_root = True
    os.chdir(project_root)

# Imports own modules.
#
import scripts.Visualization as Visualization
import scripts.ModelTrainer as ModelTrainer
import scripts.Utils as utils
import scripts.Simulation_config as Simulation_config
from scripts.Simulation_config import *
import scripts.Model as Model
import scripts.ModelAdapter as ModelAdapter


/home/molu/miniconda3/envs/xlstm/lib/python3.11/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /opt/conda/conda-bld/pytorch_1704987288773/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
No CUDA runtime is found, using CUDA_HOME='/home/molu/miniconda3/envs/xlstm'


In [2]:
importlib.reload(Model)
importlib.reload(ModelTrainer)
importlib.reload(Simulation_config)
importlib.reload(utils)

# Run the whole simulation
#
train_all_models = False
if train_all_models:
    configs = Simulation_config.configs
    ModelTrainer.ModelTrainer().run(configs)

# Print the sumarized simulation results
#
utils.Evaluate_Models.print_results('scripts/outputs/all_train_histories.pkl')


defaultdict(<function scripts.Utils.Evaluate_Models.print_results.<locals>.<lambda>()>,
            {Config_of_one_run(modelSize='MEDIUM', doPretraining=True, doTransferLearning=True, aggregation_Count='data/london_loadprofiles_50households_each.pkl', nrOfComunities=20, trainingHistory=365, testSize=92, trainingFuture=0, devSize=58, usedModels=('SyntheticLoadProfile', 'KNN', 'PersistencePrediction', 'xLSTM', 'LSTM', 'Transformer'), epochs=100): defaultdict(<function scripts.Utils.Evaluate_Models.print_results.<locals>.<lambda>.<locals>.<lambda>()>,
                         {'SyntheticLoadProfile': [21.72661994286418,
                           22.625958118362274,
                           23.135663600869105,
                           20.29830888967362,
                           21.251464932662554,
                           21.867396322971082,
                           19.37890778252043,
                           24.716417482065864,
                           19.891284128719818,
 

In [91]:
# Plot the losses over the training.
# (To see, if the models are trained stable.)
#
importlib.reload(utils)

plot_only_single_config = True
plotted_config =     Config_of_one_run(ModelSize.MEDIUM, DoPretraining.YES, DoTransferLearning.YES, Aggregation_Count._50_HOUSEHOLDS, NrOfComunities._20, 
            TrainingHistory._12_MONTH, TestSize._3_MONTH, TrainingFuture._0_MONTH, DevSize._2_MONTH, UsedModels.ALL, Epochs.DEFAULT)


utils.Evaluate_Models.plot_training_losses_over_epochs('scripts/outputs/all_train_histories.pkl',
                                                       plot_only_single_config,
                                                       plotted_config
                                                       )


Plotted Config:
("Config_of_one_run(modelSize='MEDIUM', doPretraining=True, "
 'doTransferLearning=True, '
 "aggregation_Count='data/london_loadprofiles_50households_each.pkl', "
 'nrOfComunities=20, trainingHistory=365, testSize=92, trainingFuture=0, '
 "devSize=58, usedModels=('SyntheticLoadProfile', 'KNN', "
 "'PersistencePrediction', 'xLSTM', 'LSTM', 'Transformer'), epochs=100)")


In [5]:
# Evaluate specific model of given power profiles and given configurations
#

importlib.reload(Visualization)
importlib.reload(Model)
importlib.reload(utils)
importlib.reload(ModelAdapter)

# Define a specific configuration
#
model_type = 'xLSTM'
community_id = 0  # chose one of many energy communites
myConfig = Config_of_one_run(ModelSize.MEDIUM, DoPretraining.YES, DoTransferLearning.YES, Aggregation_Count._50_HOUSEHOLDS, NrOfComunities._20, 
            TrainingHistory._12_MONTH, TestSize._3_MONTH, TrainingFuture._0_MONTH, DevSize._2_MONTH, UsedModels.ALL, Epochs.DEFAULT)

modelTrainer = ModelTrainer.ModelTrainer()
loadProfiles, weatherData, public_holidays_timestamps = modelTrainer.load_data(myConfig)
modelAdapter = ModelAdapter.ModelAdapter(public_holidays_timestamps, 
                                            trainHistory = myConfig.trainingHistory,
                                            testSize = myConfig.testSize, 
                                            trainFuture = myConfig.trainingFuture, 
                                            devSize = myConfig.devSize, 
                                            )
X, Y = modelAdapter.transformData(loadProfiles[community_id], weatherData)
path_to_trained_parameters = 'scripts/outputs/all_trained_models.pth'
num_of_features = X['train'].shape[2]
P_el_predicted = modelAdapter.deNormalizeY(Y['test']).flatten().unsqueeze(0)
test_profile = f"scripts/outputs/file_{community_id}.pkl"
my_Model = utils.Deserialize.get_trained_model(path_to_trained_parameters, model_type, test_profile, 
                                                myConfig, num_of_features, modelAdapter)

# Plot the chosen model
#
plotlyApp = Visualization.PlotlyApp(X, Y, my_Model, modelAdapter, None, 'UTC')
plotlyApp.run(myport=8050)


/home/molu/miniconda3/envs/xlstm/lib/python3.11/site-packages/meteostat/interface/meteodata.py:167: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



In [89]:
# Print the model parameter sizes
#

importlib.reload(Model)

model_types = ['xLSTM', 'LSTM', 'Transformer']
for model_type in model_types:
    for model_size in ['SMALL', 'MEDIUM', 'LARGE']:

        # Test, if the model is run-able
        num_of_features = 18
        m = Model.Model(model_type, model_size, num_of_features)
        x = torch.zeros((7, 24, num_of_features))
        m.my_model(x)

        # Print the model's parameter count
        nr_of_parameters = m.get_nr_of_parameters(do_print=False)
        print(f"{model_type} - {model_size} has {nr_of_parameters} parameters.")


xLSTM - SMALL has 19783 parameters.
xLSTM - MEDIUM has 37655 parameters.
xLSTM - LARGE has 84407 parameters.
LSTM - SMALL has 19031 parameters.
LSTM - MEDIUM has 40503 parameters.
LSTM - LARGE has 80639 parameters.
Transformer - SMALL has 19831 parameters.
Transformer - MEDIUM has 41791 parameters.
Transformer - LARGE has 80951 parameters.
